In [1]:
%autosave 100

import pandas as pd
df = pd.read_csv("data/constitutions.csv")

Autosaving every 100 seconds


In [2]:
countries = list(dict.fromkeys(df['country']))
countries[185] = "united-states"

In [3]:
urls = []
for i in countries:
    urls.append("https://data.worldbank.org/country/"+i)


## Scraping 

In [4]:
URL_ARTICLE = urls[1]
XPATH_URL = "//div[@class='buttonGroup']/a[@class='links btn-item icon-flag']/@href"

In [5]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_URL

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0]
        self.news_title = self.clean_text(self.news_title)

In [6]:
import random
from requests_html import HTMLSession
from requests_html import HTML
import w3lib.html
import html

scraper = Scraper()
response =scraper.get(URL_ARTICLE)

In [7]:
scraper.parse(response)

In [8]:
from tqdm.notebook import tqdm
tqdm().pandas()
id = []
info_urls = []

n = 0
for url in tqdm(urls):
    try:
        response = scraper.get(url)
        scraper.parse(response)
        #Storing
        info_urls.append(scraper.news_title)
        id.append(n)
    except IndexError:
        pass
    
    n = n+1

0it [00:00, ?it/s]

  0%|          | 0/194 [00:00<?, ?it/s]

In [9]:
print(len(id))
print(len(info_urls))

176
176


In [ ]:
n = 0
df = pd.read_html(info_urls[n], attrs = {'id': 'table-ddpreport-data'})[0]
df = df.drop(df.tail(3).index)
df = df.drop(df.head(2).index)
df = df.T
headers = df.iloc[0]
df = pd.DataFrame(df.values[1:], columns=headers)
df.drop(['People','Environment','Economy','States and markets','Global links'], axis=1, inplace=True)
df['Date'] = [1990,2000,2010,2020]
df['country'] = countries[id[n]]
n = n+1

for url in tqdm(info_urls-1):
    df1 = pd.read_html(url, attrs = {'id': 'table-ddpreport-data'})[0]
    df1 = df1.drop(df1.tail(3).index)
    df1 = df1.drop(df1.head(2).index)
    df1 = df1.T
    headers = df1.iloc[0]
    df1 = pd.DataFrame(df1.values[1:], columns=headers)
    df1.drop(['People','Environment','Economy','States and markets','Global links'], axis=1, inplace=True)
    df1['Date'] = [1990,2000,2010,2020]
    df1['country'] = countries[id[n]]
    df = pd.concat([df, df1])
    n = n+1

In [18]:
df.to_csv("data/world-bank.csv", index=False)